In [3]:
import pandas as pd 
import os 

PROJECT_PATH = os.getcwd()

In [4]:
rsi_file = f"{PROJECT_PATH}/Ticker_performance_rsi.csv"
arima_file = f"{PROJECT_PATH}/Ticker_performance_arima.csv"


df_rsi = pd.read_csv(rsi_file)

In [8]:
df_rsi

choices = {}
for row in df_rsi.itertuples():
    choices[row.ticker] = f"{row.rank} -- Performance: {'{:.2f}'.format(row.performance)}%"
 

{'WULF': '1 -- Performance: 117.72%',
 'GLTO': '2 -- Performance: 103.63%',
 'FSLR': '3 -- Performance: 97.40%',
 'ZYME': '4 -- Performance: 70.90%',
 'SSRM': '5 -- Performance: 60.55%',
 'NATR': '6 -- Performance: 53.87%',
 'ICHR': '7 -- Performance: 48.37%',
 'ATHM': '8 -- Performance: 45.64%',
 'CPA': '9 -- Performance: 42.87%',
 'THS': '10 -- Performance: 42.58%',
 'DXYN': '11 -- Performance: 41.47%',
 'TTEK': '12 -- Performance: 40.70%',
 'KIRK': '13 -- Performance: 39.02%',
 'FROG': '14 -- Performance: 38.56%',
 'HDSN': '15 -- Performance: 38.06%',
 'LICN': '16 -- Performance: 35.94%',
 'AIRT': '17 -- Performance: 35.94%',
 'NUVL': '18 -- Performance: 35.60%',
 'PAA': '19 -- Performance: 33.78%',
 'IESC': '20 -- Performance: 31.89%',
 'INDI': '21 -- Performance: 31.35%',
 'DSKE': '22 -- Performance: 30.62%',
 'WES': '23 -- Performance: 30.25%',
 'GAME': '24 -- Performance: 27.80%',
 'CNX': '25 -- Performance: 25.81%',
 'FSS': '26 -- Performance: 23.59%',
 'MLR': '27 -- Performanc

# alpaca

In [103]:
from datetime import datetime 
from pytz import timezone
tz = timezone('US/Eastern')
datetime.now(tz) 


#  'US/Eastern',
#  'US/Hawaii',
#  'US/Mountain',
#  'US/Pacific

datetime.datetime(2023, 8, 10, 19, 15, 25, 634758, tzinfo=<DstTzInfo 'US/Pacific' PDT-1 day, 17:00:00 DST>)

In [226]:
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import GetAssetsRequest 
from datetime import datetime, timedelta
from pytz import timezone 
 
#  'US/Eastern',
#  'US/Hawaii',
#  'US/Mountain',
#  'US/Pacific

API_KEY = "PKYHZ0L6101MSFJDO3QN"
SECRET_KEY = "EdqZW5yHPSRdgOOnREghIQEsvCz6O7i5LpMje0jJ"

def get_positions():    
    trading_client = TradingClient(API_KEY, SECRET_KEY, paper=True)
    positions = trading_client.get_all_positions()
    pos = [] 
    for position in positions: 
        position = dict(position)   
        pos.append((position["symbol"],position["exchange"].value, position["avg_entry_price"], position["current_price"],
            position["qty"] , position["side"].value ))
    
    return pd.DataFrame(pos, columns = ["Ticker", "Market", "Avg_Entry_Price", "Current_Price", "Qty", "Side"])

    #pp = get_positions()

def get_account():
    account = trading_client.get_account()
    account = dict(account)
    return account["account_number"], account["buying_power"], account["portfolio_value"] 


def get_history(ticker, interval,  data_points=250): 
    """ 
        intervals: 59Min, 23Hour, 1Day, 1Week, 12Month .
        from_time, to_time. to_time default to None. in format of "2023-02-12 12:00"
        minimum requires 250 data points.take some extra
        for precise, need to write a function
    """  
    ticker = ticker.upper()
    interval = '1Day' if interval=='1d' else '1Hour' if interval=='1h' else '5Min' if interval=='5m' else '15Min'

    if interval== '1Day':
        from_time = datetime.utcnow() + timedelta(days=-450)
        data_points = 300 
    elif interval == '1Hour':
        from_time = datetime.utcnow() + timedelta(days=-20)
        data_points = 20*24 
    elif interval == '5Min':
        from_time = datetime.utcnow() + timedelta(days=-6)
        data_points = 16 * 12 * 6
    else: #15in
        from_time = datetime.utcnow() + timedelta(days=-15)
        data_points = 16 * 4 * 15

    from_time = from_time.strftime('%Y-%m-%d')  
    ticker = ticker.upper()  
    api = tradeapi.REST(API_KEY  ,SECRET_KEY, END_POINT)
    stock_data = api.get_bars(ticker, timeframe=interval, start=from_time, limit=data_points).df
    stock_data.reset_index(inplace=True)
    #stock_data = api.get_bars(ticker, timeframe="1Day", start="2023-02-01", limit=250).df 
    stock_data.columns = ["Date", "Open", "High", "Low", "Close", "Volume", "Trades", "VWap"] 

    return stock_data 
 
def get_history_brief(ticker, interval,  data_points=250): 
    """ 
        intervals: 59Min, 23Hour, 1Day, 1Week, 12Month .
        from_time, to_time. to_time default to None. in format of "2023-02-12T12:00"
    """  
    ticker = ticker.upper()
    interval = '1Day' if interval=='1d' else '1Hour' if interval=='1h' else '5Min' if interval=='5m' else '15Min'
    ticker = ticker.upper()  
    api = tradeapi.REST(API_KEY  ,SECRET_KEY, END_POINT)
    stock_data = api.get_bars(ticker, timeframe=interval,  limit=data_points).df
    stock_data.reset_index(inplace=True) 
    #stock_data = api.get_bars(ticker, timeframe="1Day", start="2023-02-01", limit=250).df 
    stock_data.columns = ["Date", "Open", "High", "Low", "Close", "Volume", "Trades", "VWap"] 
    return stock_data 

def get_orders(status = 'closed'):
    # status='open' 
    orders = api.list_orders(
        status=status,
        limit=100,
        nested=True  # show nested multi-leg orders
    )

    results = []
    for oo in orders:       
        results.append((oo.client_order_id, oo.symbol, oo.created_at,oo.filled_at, oo.filled_avg_price, \
            oo.filled_qty, oo.id, oo.order_type, oo.status))
    
    return pd.DataFrame(results, columns=["client_order_id", 'ticker', "created_at", "filled_at", "filled_avg_price",\
                    "filled_qty",   "id", "order_type", "status"])

def cancel_order(order_id):
    """
        order_id: client order id
    """
    api.cancel_order(order_id = order_id )        
    return 0 
    
    
def submit_order(order_type):
    """
        order types: 
            MarketOrder
            GTC
            Limit
    """
    # preparing market order
    market_order_data = MarketOrderRequest(
                        symbol="SPY",
                        qty=0.023,
                        side=OrderSide.BUY,
                        time_in_force=TimeInForce.DAY
                        )

    # Market order
    market_order = trading_client.submit_order(
                    order_data=market_order_data
                )

    # preparing limit order
    limit_order_data = LimitOrderRequest(
                        symbol="BTC/USD",
                        limit_price=17000,
                        notional=4000,
                        side=OrderSide.SELL,
                        time_in_force=TimeInForce.FOK
                    )

    # Limit order
    limit_order = trading_client.submit_order(
                    order_data=limit_order_data
                )
                    


In [229]:
get_history ("AAPL", "1d",  data_points=250)

,Date,Open,High,Low,Close,Volume,Trades,VWap
0,2022-05-18 04:00:00+00:00,146.850,147.3601,139.900,140.82,108715044,969888,142.703174
1,2022-05-19 04:00:00+00:00,139.880,141.6600,136.600,137.35,135359835,1208776,138.666812
2,2022-05-20 04:00:00+00:00,139.090,140.7000,132.610,137.59,136313104,1122772,136.395212
3,2022-05-23 04:00:00+00:00,137.790,143.2600,137.650,143.11,117338199,846886,141.435007
4,2022-05-24 04:00:00+00:00,140.805,141.9700,137.330,140.36,103517401,806070,139.410797
...,...,...,...,...,...,...,...,...
295,2023-07-24 04:00:00+00:00,193.410,194.9100,192.250,192.75,45715465,527113,193.200451
296,2023-07-25 04:00:00+00:00,193.330,194.4400,192.915,193.62,37335438,446767,193.803513
297,2023-07-26 04:00:00+00:00,193.670,195.6400,193.320,194.50,47528567,503713,194.374221
298,2023-07-27 04:00:00+00:00,196.020,197.2000,192.550,193.22,47479389,570453,194.878894


In [221]:
oo = get_orders(status='closed') 
api.cancel_order(order_id = oid )

In [ ]:

API_KEY = "PKYHZ0L6101MSFJDO3QN"
SECRET_KEY = "EdqZW5yHPSRdgOOnREghIQEsvCz6O7i5LpMje0jJ"

In [233]:
api = tradeapi.REST( "PKYHZ0L6101MSFJDO3QN"  , "EdqZW5yHPSRdgOOnREghIQEsvCz6O7i5LpMje0jJ",  "https://paper-api.alpaca.markets")
#stock_data = api.get_bars(ticker, timeframe=interval, start=from_time, limit=data_points).df

In [234]:
api.get_bars("AAPL", timeframe="1Day", start='2022-05-18', limit=50).df

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2022-05-18 04:00:00+00:00,146.850,147.3601,139.9000,140.82,108715044,969888,142.703174
2022-05-19 04:00:00+00:00,139.880,141.6600,136.6000,137.35,135359835,1208776,138.666812
2022-05-20 04:00:00+00:00,139.090,140.7000,132.6100,137.59,136313104,1122772,136.395212
2022-05-23 04:00:00+00:00,137.790,143.2600,137.6500,143.11,117338199,846886,141.435007
2022-05-24 04:00:00+00:00,140.805,141.9700,137.3300,140.36,103517401,806070,139.410797
2022-05-25 04:00:00+00:00,138.430,141.7850,138.3400,140.52,91793616,680230,140.077911
2022-05-26 04:00:00+00:00,137.390,144.3400,137.1400,143.78,90366284,668096,142.390702
2022-05-27 04:00:00+00:00,145.375,149.6800,145.2600,149.64,90670289,683010,148.198783
2022-05-31 04:00:00+00:00,149.070,150.6600,146.8400,148.84,102805726,698608,149.095862


In [236]:
import datetime
import time
 
# Convert to UNIX timestamp (seconds since 1970-01-01 00:00:00 UTC)
timestamp_seconds = str(int(time.mktime(datetime.datetime.utcnow().timetuple())))

print(timestamp_seconds)

1691805701


In [237]:
from alpaca.trading.requests import MarketOrderRequest, LimitOrderRequest

In [238]:
help(LimitOrderRequest)

Help on class LimitOrderRequest in module alpaca.trading.requests:

class LimitOrderRequest(OrderRequest)
 |  LimitOrderRequest(*, symbol: str, qty: Optional[float] = None, notional: Optional[float] = None, side: alpaca.trading.enums.OrderSide, type: alpaca.trading.enums.OrderType, time_in_force: alpaca.trading.enums.TimeInForce, order_class: Optional[alpaca.trading.enums.OrderClass] = None, extended_hours: Optional[bool] = None, client_order_id: Optional[str] = None, take_profit: Optional[alpaca.trading.requests.TakeProfitRequest] = None, stop_loss: Optional[alpaca.trading.requests.StopLossRequest] = None, limit_price: float) -> None
 |  
 |  Used to submit a limit order.
 |  
 |  Attributes:
 |      symbol (str): The symbol identifier for the asset being traded
 |      qty (Optional[float]): The number of shares to trade. Fractional qty for stocks only with market orders.
 |      notional (Optional[float]): The base currency value of the shares to trade. For stocks, only works with M

In [241]:

from alpaca.trading.enums import OrderSide, TimeInForce, OrderType
for n in OrderType:
    print(n)

OrderType.MARKET
OrderType.LIMIT
OrderType.STOP
OrderType.STOP_LIMIT
OrderType.TRAILING_STOP


In [ ]:

# preparing market order
market_order_data = MarketOrderRequest(
                    symbol="SPY",
                    qty=0.023,
                    side=OrderSide.BUY,
                    time_in_force=TimeInForce.DAY
                    )

# Market order
market_order = trading_client.submit_order(
                order_data=market_order_data
               )

# preparing limit order
limit_order_data = LimitOrderRequest(
                    symbol="BTC/USD",
                    limit_price=17000,
                    notional=4000,
                    side=OrderSide.SELL,
                    time_in_force=TimeInForce.FOK
                   )

# Limit order
limit_order = trading_client.submit_order(
                order_data=limit_order_data
              )

In [166]:
https://alpaca.markets/docs/trading/getting_started/how-to-orders/

480

In [4]:
import pandas as pd 
dfd = pd.read_csv(r'C:\development\poc\stockai\tessssssssssssssssssss.csv')
for row in dfd.itertuples():
    print(row)
    print(row.Index)
    break

Pandas(Index=0, Date='2023-07-27 08:00:00+00:00', Open=195.42, High=196.34, Low=195.42, Close=196.2, Volume=20584, Trades=484, VWap=196.126325)
0


In [7]:
dfd

,Date,Open,High,Low,Close,Volume,Trades,VWap
0,2023-07-27 08:00:00+00:00,195.420,9999.000,195.420,196.2000,20584,484,196.126325
1,2023-07-27 08:15:00+00:00,196.190,196.260,196.140,196.2300,13239,269,196.180577
2,2023-07-27 08:30:00+00:00,196.250,196.250,196.040,196.0400,7725,179,196.097491
3,2023-07-27 08:45:00+00:00,196.040,196.200,195.980,196.1200,13705,218,196.084181
4,2023-07-27 09:00:00+00:00,196.190,196.200,196.030,196.1800,8728,139,196.097835
...,...,...,...,...,...,...,...,...
475,2023-08-07 14:45:00+00:00,178.760,178.955,178.065,178.1100,3230489,55013,178.563600
476,2023-08-07 15:00:00+00:00,178.100,178.180,177.560,177.9801,4502263,53719,177.925300
477,2023-08-07 15:15:00+00:00,177.990,178.050,177.570,177.7700,3236330,38948,177.806312
478,2023-08-07 15:30:00+00:00,177.765,178.175,177.350,178.1350,3213676,38753,177.775198


In [13]:

df_enrich = dfd.copy()
span = 14 
min_periods = 3
price_data = df_enrich['Close']
diff = price_data.diff(1).dropna()
up_chg = 0 * diff #arrays
down_chg = 0 * diff
up_chg[diff > 0] = diff[ diff>0 ]
down_chg[diff < 0] = diff[ diff < 0 ]
 
up_series_avg   = up_chg.ewm(span=span , min_periods=min_periods).mean()
down_series_avg = down_chg.ewm(span=span , min_periods=min_periods).mean()

rs = abs(up_series_avg/down_series_avg)
rsi = 100 - 100/(1+rs)

df_enrich['Up'] =up_series_avg
df_enrich['Down'] =down_series_avg
df_enrich[f'RS_{span}'] =rs
df_enrich[f'RSI_{span}'] =rsi

list(df_enrich[f'RSI_{span}'])[-1]

38.223242994055205

In [14]:
df = dfd.copy()
price_type ='Close'
price_data = df[price_type]
diff = price_data.diff(1).dropna()
up_chg = 0 * diff #arrays
down_chg = 0 * diff
up_chg[diff > 0] = diff[ diff>0 ]
down_chg[diff < 0] = diff[ diff < 0 ]

up_series_avg   = up_chg.ewm(span=span , min_periods=min_periods).mean()
down_series_avg = down_chg.ewm(span=span , min_periods=min_periods).mean()

rs = abs(up_series_avg/down_series_avg)
rsi = 100 - 100/(1+rs)

df['Up'] =up_series_avg
df['Down'] =down_series_avg
df[f'RS_{span}'] =rs
df[f'RSI_{span}'] =rsi

list(df[f'RSI_{span}'])[-1] 

38.223242994055205

In [119]:
 
orig_series = pd.Series([1, 2, 3, 5, 8, 12, 21, 23]) # Example original time series
dff = pd.DataFrame({"v0": [1, 2, 3, 5, 8, 12, 21, 23]}) 

In [123]:
orig_series

0     1
1     2
2     3
3     5
4     8
5    12
6    21
7    23
Name: v0, dtype: int64

In [143]:
dff

,v0,v1,v2
0,1,NaN,NaN
1,2,1.0,NaN
2,3,1.0,0.0
3,5,2.0,1.0
4,8,3.0,1.0
5,12,4.0,1.0
6,21,9.0,5.0
7,23,2.0,-7.0


# Arima

In [174]:

dff = pd.DataFrame({"v0": [1, 2, 3, 5, 8, 12, 21, 23]}) 
diff_values = [10, 9, 2, -3, 4]  # Example predicted values after second order differencing
d = 0
 
for i in range(1, d+1): 
    dff[f"v{i}"] = dff[f"v{i-1}"].diff(1) 
 
starts = list(reversed(dff.iloc[-1]))

values = []
values.append(starts)
 
for f in diff_values:
    row = [f]
    for v in starts[1:]: 
        value = f + v  
        row.append(value) 
        f = value  
    values.append(row)
    starts = row 


forecasts = [v[-1] for v in values]

forecasts[1:] 

[10, 9, 2, -3, 4]

In [173]:
dff

,v0,v1
0,1,NaN
1,2,1.0
2,3,1.0
3,5,2.0
4,8,3.0
5,12,4.0
6,21,9.0
7,23,2.0


In [148]:
dff.tail(1)

,v0,v1,v2
7,23,2.0,-7.0


In [175]:
dd =  [79.99448277612142, 79.97896555224283, 79.96344832836424, 79.94793110448565, 79.93241388060706]
dd 

[79.99448277612142,
 79.97896555224283,
 79.96344832836424,
 79.94793110448565,
 79.93241388060706]

In [176]:
col = [ "c1", "c2", "c3", "c4", "c5"] 

col

['c1', 'c2', 'c3', 'c4', 'c5']

In [183]:
dd2 = ((d, )for d in dd )
dd2

<generator object <genexpr> at 0x000002011BADFE40>

In [184]:
pd.DataFrame(data=dd2, columns=col )

ValueError: 5 columns passed, passed data had 1 columns

In [103]:

 
news = []
for v in diff_values:    
    new_row = []
    base = 0 
    for i in range(d, -1, -1): 
        
        if i == d:
            print('   darray[d] ', darray[d])
            new_row.append(darray[d]+v)
            base = darray[d]+v
        else:
            new_row.insert(0, base +darray[d] )
            base = base +darray[d] 
    darray = new_row 
    news.append(new_row)


#darray

   darray[d]  -7.0
   darray[d]  3.0
   darray[d]  12.0
   darray[d]  14.0
   darray[d]  11.0


In [102]:
news

[[-11.0, -4.0, 3.0],
 [18.0, 15.0, 12.0],
 [38.0, 26.0, 14.0],
 [39.0, 25.0, 11.0],
 [37.0, 26.0, 15.0]]

In [104]:
darray

[37.0, 26.0, 15.0]

In [47]:
aaa = aaa[-10:]

pandas.core.series.Series

In [50]:
ff = pd.DataFrame({"v":[3,4,5,6,4,3,2,1,11,13]})

In [53]:
ff['a'] = ff['v'].diff(1)

In [55]:
origin

,v,a,b
0,3,NaN,NaN
1,4,1.0,NaN
2,5,1.0,0.0
3,6,1.0,0.0
4,4,-2.0,-3.0
5,3,-1.0,1.0
6,2,-1.0,0.0
7,1,-1.0,0.0
8,11,10.0,11.0
9,13,2.0,-8.0


14

In [113]:
fbc

,Date,Open,High,Low,Close,Volume,Trades,VWap
0,2023-08-10 04:00:00+00:00,179.48,180.75,177.6,177.97,54361628,648242,178.795943


In [22]:
import alpaca_trade_api as tradeapi
END_POINT = "https://paper-api.alpaca.markets"

api = tradeapi.REST(API_KEY  ,SECRET_KEY, END_POINT  )

# Get the last 100 closed orders
closed_orders = api.list_orders(
    status='closed',
    limit=100,
    nested=True  # show nested multi-leg orders
)

api.submit_order(
    symbol=symbol,
    qty=1,
    side='buy',
    type='market',
    time_in_force='gtc',
    order_class='bracket',
    stop_loss={'stop_price': symbol_price * 0.95,
               'limit_price':  symbol_price * 0.94},
    take_profit={'limit_price': symbol_price * 1.05}
)

In [24]:
portfolio = api.list_positions()

In [25]:
portfolio

[Position({   'asset_class': 'us_equity',
     'asset_id': 'b0b6dd9d-8b9b-48a9-ba46-b9d54906e415',
     'asset_marginable': True,
     'avg_entry_price': '189.3',
     'change_today': '0',
     'cost_basis': '-189.3',
     'current_price': '178.19',
     'exchange': 'NASDAQ',
     'lastday_price': '178.19',
     'market_value': '-178.19',
     'qty': '-1',
     'qty_available': '-1',
     'side': 'short',
     'symbol': 'AAPL',
     'unrealized_intraday_pl': '0',
     'unrealized_intraday_plpc': '0',
     'unrealized_pl': '11.11',
     'unrealized_plpc': '0.0586899101954569'}),
 Position({   'asset_class': 'us_equity',
     'asset_id': 'b28f4066-5c6d-479b-a2af-85dc1a8f16fb',
     'asset_marginable': True,
     'avg_entry_price': '424.2175',
     'change_today': '0',
     'cost_basis': '1696.87',
     'current_price': '445.75',
     'exchange': 'ARCA',
     'lastday_price': '445.75',
     'market_value': '1783',
     'qty': '4',
     'qty_available': '4',
     'side': 'long',
     'symb

In [40]:
stock_data = api.get_bars("AAPL", timeframe="1Day", start="2023-02-01", limit=250).df

In [53]:
stock_data = api.get_bars("AAPL", timeframe="5min",  start="2023-08-01", limit=250).df

In [57]:
stock_data = api.get_bars("AAPL", timeframe="1Hour",  start="2023-06-01", limit=250).df
stock_data.shape

(250, 7)

# 59Min, 23Hour, 1Day, 1Week, 12Month 

In [47]:
stock_data

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2020-01-02 05:00:00+00:00,296.24,300.6000,295.190,300.35,33912668,283487,298.439523
2020-01-03 05:00:00+00:00,297.15,300.5800,296.500,297.43,36633895,280941,298.810385
2020-01-06 05:00:00+00:00,293.79,299.9600,292.750,299.80,29644774,242866,297.799601
2020-01-07 05:00:00+00:00,299.84,300.9000,297.480,298.39,27877668,227869,298.924297
2020-01-08 05:00:00+00:00,297.16,304.4399,297.156,303.19,33090947,261771,301.165488
...,...,...,...,...,...,...,...
2020-12-21 05:00:00+00:00,125.02,128.3100,123.449,128.23,121252029,746689,126.334071
2020-12-22 05:00:00+00:00,131.61,134.4050,129.650,131.88,169352325,1132118,132.251410
2020-12-23 05:00:00+00:00,132.16,132.4300,130.780,130.96,88238756,557223,131.598436


In [59]:
df = stock_data.copy()
df.reset_index(inplace=True)

In [61]:
df.columns = ["Date", "Open", "High", "Low", "Close", "Volume", "Trades", "VWap"]

,Date,Open,High,Low,Close,Volume,Trades,VWap
0,2023-06-01 08:00:00+00:00,177.8000,177.95,177.350,177.9500,27285,738,177.625059
1,2023-06-01 09:00:00+00:00,177.9400,177.94,177.560,177.7300,26675,524,177.694505
2,2023-06-01 10:00:00+00:00,177.7200,177.93,177.720,177.7500,21571,354,177.845066
3,2023-06-01 11:00:00+00:00,177.7600,178.18,177.670,177.8300,74346,1572,177.914875
4,2023-06-01 12:00:00+00:00,177.8403,178.18,177.060,177.6300,262114,5284,177.688412
...,...,...,...,...,...,...,...,...
245,2023-06-23 13:00:00+00:00,185.3800,186.18,185.010,185.2800,6740783,100671,185.531312
246,2023-06-23 14:00:00+00:00,185.2700,186.35,185.010,186.2700,6728449,139474,185.799090
247,2023-06-23 15:00:00+00:00,186.2800,186.69,185.910,186.5613,4567829,51423,186.295678
248,2023-06-23 16:00:00+00:00,186.5700,187.14,186.535,186.9830,4886050,48496,186.909244
